In [33]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision import datasets, models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image


In [34]:
# Параметры
data_dir = '/content/drive/MyDrive/Colab Notebooks/colors'  # Путь к папке с данными
num_classes = 12  # Количество классов (цветов)
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Преобразования для данных
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [35]:
# Загрузка данных
dataset = datasets.ImageFolder(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [36]:
# Определение модели
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [37]:
# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Обнуление градиентов
        outputs = model(images)  # Прямой проход
        loss = criterion(outputs, labels)  # Вычисление потерь
        loss.backward()  # Обратный проход
        optimizer.step()  # Обновление параметров
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch [1/10], Loss: 1.6372
Epoch [2/10], Loss: 0.7813
Epoch [3/10], Loss: 0.6631
Epoch [4/10], Loss: 0.5245
Epoch [5/10], Loss: 0.5711
Epoch [6/10], Loss: 0.4777
Epoch [7/10], Loss: 0.4168
Epoch [8/10], Loss: 0.3736
Epoch [9/10], Loss: 0.3662
Epoch [10/10], Loss: 0.3645


In [38]:
# Оценка модели
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


In [39]:
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 75.33%


In [40]:
# Сохранение модели
torch.save(model.state_dict(), 'color_classifier.pth')

In [41]:
# Функция для предсказания цвета на новом изображении
def predict_color(image_path):
    model.eval()
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Добавляем размерность для батча
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    return dataset.classes[predicted.item()]

In [42]:
# Пример использования функции предсказания
color = predict_color('/content/drive/MyDrive/Colab Notebooks/colors/final.jpg')
print(f'Predicted color: {color}')

Predicted color: Pink
